In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import pandas as pd
import urllib.request
import numpy as np
import matplotlib as plt
from IPython.display import display
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import scipy.sparse as sp
from sklearn import metrics

In [ ]:
# read in the file and convert to sparse matrix
cf = open('pancancer_mutations_merged.csv', 'r')
pat_out = OrderedDict()
gene_idx = dict()
gene_counter = 0
cols = cf.readline().split(",")
labels_d = OrderedDict()
ignored = set()
for i, line in enumerate(cf):
  nl = dict(zip(cols, line.split(',')))
  biotype, pat, gene, ct = nl['BIOTYPE'], nl['bcr_patient_barcode'], nl['Hugo_Symbol'], nl['cancer_type']
  if biotype != 'protein_coding':
    ignored.add(gene)
    continue
  labels_d[pat] = ct
  if gene not in gene_idx:
    gene_idx[gene] = gene_counter
    gene_counter +=1
  if pat not in pat_out:
    pat_out[pat] = set()
  pat_out[pat].add(gene_idx[gene])

print(len(gene_idx))

In [ ]:
mat = sp.dok_matrix((len(pat_out.keys()), len(gene_idx)), dtype=np.int8)

In [ ]:
for i, (patient, genes) in enumerate(pat_out.items()):
  mat[i, list(genes)] = 1
mat = mat.tocsr()

In [ ]:
mat_d = mat.todense()
mat_d.shape

In [ ]:
labels = pd.DataFrame(labels_d.values())
shuffle = np.random.permutation(np.arange(mat_d.shape[0]))
#X, Y  = principalComponents[shuffle], labels.iloc[shuffle]
X, Y  = mat_d[shuffle], labels.iloc[shuffle]
train_X = X[:6000]
train_Y = Y[:6000][0].values

test_X = X[6000:]
test_Y = Y[6000:][0].values


In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(train_X)

In [ ]:
ev = np.cumsum(pca.explained_variance_)
print("Number of features that expain 99% of the variance", len(ev[ev<99.0]))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=113)
pca.fit(train_X)
train_data = pca.transform(train_X)
test_data = pca.transform(test_X)


# Logistic regression with PCA

In [ ]:
clf = LogisticRegression()

clf = clf.fit(train_data,  train_Y)
pred_Y = clf.predict(test_data)

print("Accuracy:",metrics.f1_score(test_Y, pred_Y, average='micro'))

# Random Forest with PCA

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=20)

clf = clf.fit(train_data, train_Y)
pred_Y = clf.predict(test_data)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.f1_score(test_Y, pred_Y, average='micro'))

# Logistic regression with L1/L2